In [5]:
# Import necessary initial libraries
import os
import sys
import importlib
from importlib import reload

In [2]:
# Sets the root directory of the project as the working directory
os.chdir('..')

In [23]:
# Check the current working directory
os.getcwd()

'/Users/darlanmnunes/Dev/DSc_git/PhD_Thesis_Step3_OSM_Toponyms'

In [17]:
# Import necessary modules for this notebook
from libs.merge_geojson_grids import *

In [19]:
# Reload the module to ensure the latest version is used
reload(sys.modules['libs.merge_geojson_grids'])

<module 'libs.merge_geojson_grids' from '/Users/darlanmnunes/Dev/DSc_git/PhD_Thesis_Step3_OSM_Toponyms/libs/merge_geojson_grids.py'>

### Consolidação dos Arquivos GeoJSON das Grades Geoespaciais em um único arquivo

In [15]:
# Lista dos arquivos GeoJSON para merge (ordem importa - primeiro arquivo tem prioridade)
input_files = [
    "results/1_output_grid/partial_results/resultado_consolidado1.geojson",
    "results/1_output_grid/partial_results/resultado_consolidado2.geojson"
]

In [20]:
# output file path
output_path = "results/1_output_grid/partial_results/"
output_file_name = "resultado_consolidado_step1"

# Executa o merge
merge_geojson_to_gpkg(input_files, os.path.join(output_path, output_file_name))

Merge concluído! Arquivo salvo como GeoPackage em: results/1_output_grid/partial_results/resultado_consolidado_step1.gpkg
CRS mantido: EPSG:4326
Total de células únicas: 5940


### 3. Métricas de Tempo

In [21]:
# Leitura do Arquivo Excel
file_name = [
    "results/1_output_grid/time_metrics/time_log_OHSOME_requests.xlsx"
]

In [25]:
# Leitura do Arquivo Excel
df_time = pd.read_excel(file_name[0], sheet_name='step1')

In [26]:
# Exibe as primeiras linhas para validar
display(df_time.head(10))

,Lote 1: 0%| | 0/20 [00:00<?,?it/s]Ainda trabalhando...,Unnamed: 2,Unnamed: 3
0,Lote 1: 25%|██▌ | 5/20 [06:16<08:19,33.27s/it]Ainda trabalhando...,NaN,NaN
1,Lote 1: 50%|█████ | 10/20 [13:24<06:19,37.98s/it]Ainda trabalhando...,NaN,NaN
2,Ainda trabalhando...,NaN,NaN,NaN
3,Lote 1: 75%|███████▌ | 15/20 [21:42<03:43,44.72s/it]Ainda trabalhando...,NaN,NaN
4,Ainda trabalhando...,NaN,NaN,NaN
5,Lote 1: 100%|██████████| 20/20 [30:34<00:00,91.71s/it],NaN,NaN
6,[SALVO] /content/drive/MyDrive/UFPR_PPGCG_DSc/...,NaN,NaN,NaN
7,[CONSOLIDADO] resultado_consolidado.geojson at...,NaN,NaN,NaN
8,Tempo do lote 1: 30.0 min 34.31 seg,NaN,NaN,NaN
9,Lote 2: 0%| | 0/20 [00:00<?,?it/s]Ainda trabalhando...,NaN,NaN


In [27]:
# Exibe as últimas linhas para validar
display(df_time.tail(10))

,Lote 1: 0%| | 0/20 [00:00<?,?it/s]Ainda trabalhando...,Unnamed: 2,Unnamed: 3
4407,Lote 375: 50%|█████ | 10/20 [13:34<06:41,40.14s/it]Ainda trabalhando...,NaN,NaN
4408,Lote 375: 75%|███████▌ | 15/20 [20:17<03:09,37.95s/it]Ainda trabalhando...,NaN,NaN
4409,Lote 375: 95%|█████████▌| 19/20 [26:31<00:48,48.24s/it]Ainda trabalhando...,NaN,NaN
4410,Lote 375: 100%|██████████| 20/20 [26:41<00:00,80.06s/it],NaN,NaN
4411,[SALVO] /content/drive/MyDrive/UFPR_PPGCG_DSc/...,NaN,NaN,NaN
4412,[CONSOLIDADO] resultado_consolidado.geojson at...,NaN,NaN,NaN
4413,Tempo do lote 375: 26.0 min 41.56 seg,NaN,NaN,NaN
4414,Lote 376: 0%| | 0/20 [00:00<?,?it/s]Ainda trabalhando...,NaN,NaN
4415,Lote 376: 25%|██▌ | 5/20 [06:59<09:19,37.29s/it]Ainda trabalhando...,NaN,NaN
4416,Lote 376: 50%|█████ | 10/20 [13:36<06:12,37.27s/it],NaN,NaN


In [28]:
# Função para extrair tempos dos lotes
def extrair_tempos(df):
    resultados = []

    for line in df.iloc[:, 0].dropna():
        match = re.search(r'Tempo do lote (\d+): (\d+\.?\d*) min (\d+\.?\d*) seg', line)
        if match:
            lote = int(match.group(1))
            minutos = float(match.group(2))
            segundos = float(match.group(3))
            total_segundos = int(minutos * 60 + segundos)
            resultados.append({
                'Lote': lote,
                'Minutos': minutos,
                'Segundos': segundos,
                'Total (s)': total_segundos,
                'Total (min)': total_segundos / 60,
                'Total (h)': total_segundos / 3600,
                'Total (dias)': total_segundos / (3600 * 24),
                'Duração (timedelta)': timedelta(seconds=total_segundos)
            })

    return pd.DataFrame(resultados)

In [29]:
# Aplicar a função para extrair os tempos
df_tempos = extrair_tempos(df_time)
df_tempos

,Lote,Minutos,Segundos,Total (s),Total (min),Total (h),Total (dias),Duração (timedelta)
0,1,30.0,34.31,1834,30.566667,0.509444,0.021227,0 days 00:30:34
1,2,31.0,23.44,1883,31.383333,0.523056,0.021794,0 days 00:31:23
2,3,27.0,41.91,1661,27.683333,0.461389,0.019225,0 days 00:27:41
3,4,28.0,20.47,1700,28.333333,0.472222,0.019676,0 days 00:28:20
4,5,25.0,27.84,1527,25.450000,0.424167,0.017674,0 days 00:25:27
...,...,...,...,...,...,...,...,...
380,371,26.0,22.32,1582,26.366667,0.439444,0.018310,0 days 00:26:22
381,372,29.0,29.37,1769,29.483333,0.491389,0.020475,0 days 00:29:29
382,373,28.0,53.41,1733,28.883333,0.481389,0.020058,0 days 00:28:53
383,374,26.0,56.58,1616,26.933333,0.448889,0.018704,0 days 00:26:56


In [37]:
# Cálculo do tempo total consolidado de todos os lotes
total_horas = df_tempos['Total (h)'].sum()
total_dias = df_tempos['Total (dias)'].sum()
print(f"Tempo Total (todos os lotes): {total_horas :.2f} horas")
print(f"Tempo Total (todos os lotes): {total_dias :.2f} dias")

Tempo Total (todos os lotes): 227.35 horas
Tempo Total (todos os lotes): 9.47 dias


In [52]:
# Gráfico de barras para visualizar os tempos de processamento
fig = px.bar(
    df_tempos,
    x='Lote',
    y='Total (min)',
    hover_data={
        'Minutos': ':.1f',
        'Segundos': ':.1f',
        'Total (h)': ':.3f'
    },
    labels={'Total (min)': 'Duração (minutos)'},
    title='Tempo de Processamento por Lote (20 células)',
)

# Cálculo do tempo total consolidado de todos os lotes
total_horas = df_tempos['Total (h)'].sum()
total_dias = df_tempos['Total (dias)'].sum()

# Adiciona anotação com a duração total
fig.add_annotation(
    xref="paper", 
    yref="paper",
    x=0.5, 
    y=-0.35,  # Ajuste a posição vertical aqui
    text=f"Duração Total: <br>({total_horas:.2f} horas / {total_dias:.2f} dias)",
    showarrow=False,
    font=dict(size=12),
    align="center"
)

# Ajustes estéticos
fig.update_traces(marker_color='royalblue')
fig.update_layout(
    title_x=0.5,
    xaxis_title='Lote',
    yaxis_title='Duração (minutos)',
    hoverlabel=dict(bgcolor="white", font_size=13),
    margin=dict(b=100)  # Aumenta a margem inferior para caber a anotação
)

fig.show()